In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import copy
import joblib

In [2]:
from py.utils import verifyDir

In [3]:
DATA_PATH = "data/PlacePulse_2/"

QSCORE_PATH = f"{DATA_PATH}/Qscores/all/Summaries/"
MAIN_OUT_PATH = "outputs/"
CACHE_PATH = f"{MAIN_OUT_PATH}/cache/"

In [4]:
verifyDir(CACHE_PATH)

### Generating samples 

In [5]:
data_dict = joblib.load(f"{MAIN_OUT_PATH}/static/data_train.joblib")
data_dict.keys()

dict_keys(['id', 'features', 'features_name', 'safety', 'label', 'latitude', 'longitude', 'city', 'country', 'continent', 'path'])

In [6]:
y = np.array(data_dict["safety"]).copy()
X = np.array(data_dict["features"]).copy()
X.shape, y.shape

((110988, 19), (110988,))

### Standard Scaler

In [7]:
from sklearn.preprocessing import StandardScaler

In [8]:
scaler = StandardScaler()

In [9]:
from sklearn.preprocessing import Normalizer

In [10]:
normalizer = Normalizer()

### Classifiers

In [11]:
from sklearn.svm import SVR, LinearSVR
from sklearn.linear_model import Ridge, LinearRegression, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

In [12]:
regularizer_value = np.logspace(-3, 3, num=7)
type_scaler = ['passthrough', scaler, normalizer]
# dim_reductor = [t_svd, pca, sparse_pca]
random_state = 42
max_iter=2000

In [13]:
lr = LinearRegression()

param_lr = {}
param_lr['scaler'] = copy.deepcopy(type_scaler)
#param_lr['dim_reductor'] = copy.deepcopy(dim_reductor)
param_lr['regressor'] = [lr]

In [14]:
lasso = Lasso(
    tol=1e-3,
    random_state=random_state,
    max_iter=max_iter,
)

param_lasso = {}
param_lasso['scaler'] = copy.deepcopy(type_scaler)
# param_ridge['dim_reductor'] = copy.deepcopy(dim_reductor)
param_lasso['regressor__alpha'] = copy.deepcopy(regularizer_value)
param_lasso['regressor'] = [lasso]

In [15]:
ridge = Ridge(
    tol=1e-3,
    random_state=random_state,
    max_iter=max_iter,
)

param_ridge = {}
param_ridge['scaler'] = copy.deepcopy(type_scaler)
# param_ridge['dim_reductor'] = copy.deepcopy(dim_reductor)
param_ridge['regressor__alpha'] = copy.deepcopy(regularizer_value)
param_ridge['regressor'] = [ridge]

In [16]:
linear_svm = LinearSVR(
    tol=1e-3,
    random_state=random_state,
    max_iter=max_iter,
)

param_linear_svm = {}
param_linear_svm['scaler'] = copy.deepcopy(type_scaler)
# param_linear_svm['dim_reductor'] = copy.deepcopy(dim_reductor)
param_linear_svm['regressor__C'] = copy.deepcopy(regularizer_value)
param_linear_svm['regressor__loss'] = ["epsilon_insensitive", "squared_epsilon_insensitive"]
param_linear_svm['regressor'] = [linear_svm]

In [17]:
svm = SVR(
    tol=1e-3,
    max_iter=max_iter,
)

param_svm = {}
param_svm['scaler'] = copy.deepcopy(type_scaler)
# param_linear_svm['dim_reductor'] = copy.deepcopy(dim_reductor)
param_svm['regressor__C'] = copy.deepcopy(regularizer_value)
param_svm['regressor__gamma'] = ["scale", "auto"]
param_svm['regressor__kernel'] = ["linear", "poly", "rbf"]
param_svm['regressor'] = [svm]

In [18]:
ds_tree = DecisionTreeRegressor(random_state=random_state)

param_tree = {}
param_tree["scaler"] = copy.deepcopy(type_scaler)
param_tree['regressor__max_features'] = [None, 'sqrt']
param_tree['regressor__max_depth'] = np.append(None, np.arange(10, 110, 10) )
param_tree['regressor__min_samples_split'] = np.arange(3, 7)
param_tree['regressor__min_samples_leaf'] = np.arange(3, 7)
param_tree['regressor__criterion'] = ["squared_error", "friedman_mse", "absolute_error", "poisson"]
param_tree['regressor'] = [ds_tree]

### Create Pipeline

In [19]:
from joblib import Memory

In [20]:
memory = Memory(
    location=CACHE_PATH,
    # verbose=1,
)

In [21]:
from sklearn.pipeline import Pipeline

In [22]:
pipeline = Pipeline(
    steps = [
        # 1. Scaler
        ('scaler', scaler),
        # 3. Model selecction
        ('regressor', lr),
    ],
    memory=memory,
)

### Metrics

In [23]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, make_scorer
from scipy.stats import pearsonr

In [24]:
def adj_r2_score(estimator, X, y_true):
    n, p = X.shape
    pred = estimator.predict(X)
    return 1 - ((1 - r2_score(y_true, pred)) * (n - 1))/(n-p-1)

In [25]:
def pearson_score(y_true, y_pred):
    corr, _ = pearsonr(y_true, y_pred)
    return corr

### Create GridSearch

In [26]:
from sklearn.model_selection import GridSearchCV

In [27]:
param_grid = [param_lr, 
              param_lasso,
              param_ridge,
              param_linear_svm,
              param_svm,
              #param_tree,
             ]

In [28]:
param_grid

[{'scaler': ['passthrough', StandardScaler(), Normalizer()],
  'regressor': [LinearRegression()]},
 {'scaler': ['passthrough', StandardScaler(), Normalizer()],
  'regressor__alpha': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03]),
  'regressor': [Lasso(max_iter=2000, random_state=42, tol=0.001)]},
 {'scaler': ['passthrough', StandardScaler(), Normalizer()],
  'regressor__alpha': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03]),
  'regressor': [Ridge(max_iter=2000, random_state=42, tol=0.001)]},
 {'scaler': ['passthrough', StandardScaler(), Normalizer()],
  'regressor__C': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03]),
  'regressor__loss': ['epsilon_insensitive', 'squared_epsilon_insensitive'],
  'regressor': [LinearSVR(max_iter=2000, random_state=42, tol=0.001)]},
 {'scaler': ['passthrough', StandardScaler(), Normalizer()],
  'regressor__C': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03]),
  'regressor__gamma': ['scale', 

In [29]:
grid_search = GridSearchCV(
            estimator=pipeline,
            param_grid=param_grid,
            scoring=make_scorer(pearson_score),
            refit=True,
            cv=5,  # << Use time series
            verbose=4,
)

In [30]:
grid_search

GridSearchCV(cv=5,
             estimator=Pipeline(memory=Memory(location=outputs//cache/joblib),
                                steps=[('scaler', StandardScaler()),
                                       ('regressor', LinearRegression())]),
             param_grid=[{'regressor': [LinearRegression()],
                          'scaler': ['passthrough', StandardScaler(),
                                     Normalizer()]},
                         {'regressor': [Lasso(max_iter=2000, random_state=42,
                                              tol=0.001)],
                          'regressor__alpha': array([1.e-03,...
                                              'squared_epsilon_insensitive'],
                          'scaler': ['passthrough', StandardScaler(),
                                     Normalizer()]},
                         {'regressor': [SVR(max_iter=2000)],
                          'regressor__C': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03]),
                          'regressor__gamma': ['scale', 'auto'],
                          'regressor__kernel': ['linear', 'poly', 'rbf'],
                          'scaler': ['passthrough', StandardScaler(),
                                     Normalizer()]}],
             scoring=make_scorer(pearson_score), verbose=4)

### Data Split

In [31]:
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

In [32]:
xtrain, xtest, ytrain, ytest = train_test_split(X, 
                                                y, 
                                                train_size=0.75, 
                                                random_state=random_state,
                                               )
print(xtrain.shape, ytrain.shape, xtest.shape, ytest.shape)

(83241, 19) (83241,) (27747, 19) (27747,)


### Training

In [33]:
%%time
regressor = grid_search.fit(xtrain, ytrain)

Fitting 5 folds for each of 213 candidates, totalling 1065 fits
[CV 1/5] END regressor=LinearRegression(), scaler=passthrough;, score=0.279 total time=   0.0s
[CV 2/5] END regressor=LinearRegression(), scaler=passthrough;, score=0.278 total time=   0.0s
[CV 3/5] END regressor=LinearRegression(), scaler=passthrough;, score=0.292 total time=   0.0s
[CV 4/5] END regressor=LinearRegression(), scaler=passthrough;, score=0.283 total time=   0.0s
[CV 5/5] END regressor=LinearRegression(), scaler=passthrough;, score=0.283 total time=   0.0s
________________________________________________________________________________
[Memory] Calling sklearn.pipeline._fit_transform_one...
_fit_transform_one(StandardScaler(), array([[6.135559e+00, ..., 5.086263e-04],
       ...,
       [4.656982e+00, ..., 1.734416e-01]]), 
array([5.638165, ..., 5.646758]), None, message_clsname='Pipeline', message=None)
________________________________________________fit_transform_one - 0.1s, 0.0min
[CV 1/5] END regressor=Li

[CV 1/5] END regressor=Lasso(max_iter=2000, random_state=42, tol=0.001), regressor__alpha=0.01, scaler=StandardScaler();, score=0.279 total time=   0.1s
[CV 2/5] END regressor=Lasso(max_iter=2000, random_state=42, tol=0.001), regressor__alpha=0.01, scaler=StandardScaler();, score=0.278 total time=   0.1s
[CV 3/5] END regressor=Lasso(max_iter=2000, random_state=42, tol=0.001), regressor__alpha=0.01, scaler=StandardScaler();, score=0.290 total time=   0.1s
[CV 4/5] END regressor=Lasso(max_iter=2000, random_state=42, tol=0.001), regressor__alpha=0.01, scaler=StandardScaler();, score=0.283 total time=   0.1s
[CV 5/5] END regressor=Lasso(max_iter=2000, random_state=42, tol=0.001), regressor__alpha=0.01, scaler=StandardScaler();, score=0.281 total time=   0.1s
[CV 1/5] END regressor=Lasso(max_iter=2000, random_state=42, tol=0.001), regressor__alpha=0.01, scaler=Normalizer();, score=0.220 total time=   0.0s
[CV 2/5] END regressor=Lasso(max_iter=2000, random_state=42, tol=0.001), regressor__al

/home/felipe/anaconda3/envs/server_PlacePulse/lib/python3.8/site-packages/scipy/stats/_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
/home/felipe/anaconda3/envs/server_PlacePulse/lib/python3.8/site-packages/scipy/stats/_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
/home/felipe/anaconda3/envs/server_PlacePulse/lib/python3.8/site-packages/scipy/stats/_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
/home/felipe/anaconda3/envs/server_PlacePulse/lib/python3.8/site-packages/scipy/stats/_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
/home/fe

[CV 5/5] END regressor=Lasso(max_iter=2000, random_state=42, tol=0.001), regressor__alpha=0.1, scaler=Normalizer();, score=nan total time=   0.0s
[CV 1/5] END regressor=Lasso(max_iter=2000, random_state=42, tol=0.001), regressor__alpha=1.0, scaler=passthrough;, score=0.177 total time=   0.0s
[CV 2/5] END regressor=Lasso(max_iter=2000, random_state=42, tol=0.001), regressor__alpha=1.0, scaler=passthrough;, score=0.174 total time=   0.0s
[CV 3/5] END regressor=Lasso(max_iter=2000, random_state=42, tol=0.001), regressor__alpha=1.0, scaler=passthrough;, score=0.171 total time=   0.0s
[CV 4/5] END regressor=Lasso(max_iter=2000, random_state=42, tol=0.001), regressor__alpha=1.0, scaler=passthrough;, score=0.168 total time=   0.0s
[CV 5/5] END regressor=Lasso(max_iter=2000, random_state=42, tol=0.001), regressor__alpha=1.0, scaler=passthrough;, score=0.178 total time=   0.0s
[CV 1/5] END regressor=Lasso(max_iter=2000, random_state=42, tol=0.001), regressor__alpha=1.0, scaler=StandardScaler();

/home/felipe/anaconda3/envs/server_PlacePulse/lib/python3.8/site-packages/scipy/stats/_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
/home/felipe/anaconda3/envs/server_PlacePulse/lib/python3.8/site-packages/scipy/stats/_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
/home/felipe/anaconda3/envs/server_PlacePulse/lib/python3.8/site-packages/scipy/stats/_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
/home/felipe/anaconda3/envs/server_PlacePulse/lib/python3.8/site-packages/scipy/stats/_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
/home/fe

...

/home/felipe/anaconda3/envs/server_PlacePulse/lib/python3.8/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=2000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


[CV 4/5] END regressor=SVR(max_iter=2000), regressor__C=1000.0, regressor__gamma=auto, regressor__kernel=rbf, scaler=Normalizer();, score=0.046 total time=   7.0s


/home/felipe/anaconda3/envs/server_PlacePulse/lib/python3.8/site-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=2000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


[CV 5/5] END regressor=SVR(max_iter=2000), regressor__C=1000.0, regressor__gamma=auto, regressor__kernel=rbf, scaler=Normalizer();, score=0.007 total time=   7.3s
________________________________________________________________________________
[Memory] Calling sklearn.pipeline._fit_transform_one...
_fit_transform_one(Normalizer(), array([[36.006673, ...,  0.      ],
       ...,
       [ 4.656982, ...,  0.173442]]), 
array([4.576042, ..., 5.646758]), None, message_clsname='Pipeline', message=None)
________________________________________________fit_transform_one - 0.1s, 0.0min
CPU times: user 1h 9min 20s, sys: 36.5 s, total: 1h 9min 56s
Wall time: 1h 9min 54s


/home/felipe/anaconda3/envs/server_PlacePulse/lib/python3.8/site-packages/sklearn/model_selection/_search.py:952: UserWarning: One or more of the test scores are non-finite: [ 0.28301654  0.28301654  0.29125323  0.28311033  0.28304966  0.28595634
  0.28231215  0.28214038  0.22190875  0.27152742  0.21099096         nan
  0.17376307         nan         nan         nan         nan         nan
         nan         nan         nan         nan         nan         nan
  0.28301654  0.28301654  0.29125335  0.28301654  0.28301654  0.29125444
  0.28301655  0.28301654  0.29126396  0.28301659  0.28301656  0.29129417
  0.28301704  0.28301675  0.29069115  0.28302134  0.28301795  0.28637864
  0.28304737  0.28298111  0.26484512  0.24551422  0.28299554  0.24475013
  0.25568956  0.28299426  0.26588793  0.27132975  0.28243565  0.28311775
  0.28006681  0.28294874  0.28789509  0.28049223  0.28167517  0.29032863
  0.28067559  0.28301331  0.29107965  0.22166808  0.28228149  0.29102796
  0.20683126  0.2828055

### Results

In [34]:
pd.DataFrame(grid_search.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_regressor,param_scaler,param_regressor__alpha,param_regressor__C,param_regressor__loss,param_regressor__gamma,param_regressor__kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.022693,0.010972,0.001284,0.000109,LinearRegression(),passthrough,NaN,NaN,NaN,NaN,NaN,"{'regressor': LinearRegression(), 'scaler': 'p...",0.278745,0.277893,0.291919,0.283281,0.283244,0.283017,0.004977,34
1,0.090694,0.016662,0.001826,0.000052,LinearRegression(),StandardScaler(),NaN,NaN,NaN,NaN,NaN,"{'regressor': LinearRegression(), 'scaler': St...",0.278745,0.277893,0.291919,0.283281,0.283244,0.283017,0.004977,34
2,0.075776,0.001581,0.001937,0.000101,LinearRegression(),Normalizer(),NaN,NaN,NaN,NaN,NaN,"{'regressor': LinearRegression(), 'scaler': No...",0.285458,0.286501,0.299206,0.294231,0.290870,0.291253,0.005068,7
3,0.079843,0.004553,0.001267,0.000195,"Lasso(max_iter=2000, random_state=42, tol=0.001)",passthrough,0.001,NaN,NaN,NaN,NaN,"{'regressor': Lasso(max_iter=2000, random_stat...",0.278788,0.278229,0.291961,0.283287,0.283287,0.283110,0.004918,19
4,0.066362,0.002637,0.002272,0.000419,"Lasso(max_iter=2000, random_state=42, tol=0.001)",StandardScaler(),0.001,NaN,NaN,NaN,NaN,"{'regressor': Lasso(max_iter=2000, random_stat...",0.278875,0.278060,0.291796,0.283314,0.283203,0.283050,0.004877,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208,2.136824,0.040868,0.285588,0.007154,SVR(max_iter=2000),StandardScaler(),NaN,1000.0,NaN,auto,poly,"{'regressor': SVR(max_iter=2000), 'regressor__...",-0.024594,-0.023433,-0.016027,-0.016209,-0.004185,-0.016890,0.007277,168
209,4.559080,0.044669,0.843810,0.008547,SVR(max_iter=2000),Normalizer(),NaN,1000.0,NaN,auto,poly,"{'regressor': SVR(max_iter=2000), 'regressor__...",0.132069,0.161173,0.149007,0.170937,0.155846,0.153806,0.013019,78
210,6.289460,0.228540,1.580043,0.047011,SVR(max_iter=2000),passthrough,NaN,1000.0,NaN,auto,rbf,"{'regressor': SVR(max_iter=2000), 'regressor__...",0.082122,0.109952,0.099491,0.103456,0.107038,0.100412,0.009793,114
211,4.391822,0.140587,1.271624,0.050374,SVR(max_iter=2000),StandardScaler(),NaN,1000.0,NaN,auto,rbf,"{'regressor': SVR(max_iter=2000), 'regressor__...",-0.066848,0.032642,-0.034800,-0.068470,0.051361,-0.017223,0.050176,170


In [35]:
grid_search.best_estimator_

Pipeline(memory=Memory(location=outputs//cache/joblib),
         steps=[('scaler', Normalizer()),
                ('regressor',
                 Ridge(max_iter=2000, random_state=42, tol=0.001))])

In [49]:
y_pred = regressor.predict(X)
y_pred.shape

(110988,)

In [50]:
pearsonr(y, y_pred)

PearsonRResult(statistic=0.29432369014029225, pvalue=0.0)

In [37]:
data_dict["prediction"] = y_pred.copy()

In [38]:
joblib.dump(data_dict, f"{MAIN_OUT_PATH}/static/data_test.joblib")

['outputs//static/data_test.joblib']

In [39]:
os.exit()

NameError: name 'os' is not defined

In [ ]:
print(classification_report(
    ytest,
    grid_search.predict(xtest),
    digits=4,
))

### Feature Importance

In [ ]:
importance_classes = data_df.columns[1:-3]
importance_classes.values

In [ ]:
importance = grid_search.best_estimator_._final_estimator.coef_[0]

In [ ]:
feat_importances = pd.Series(importance)
feat_importances = feat_importances.reset_index().rename(columns={"index":"class", 0:"importance"}).copy()
feat_importances["class"] = importance_classes.values
feat_importances.set_index("class", inplace=True)
feat_importances.sort_values(by="importance", inplace=True, ascending=False)
feat_importances

In [ ]:
feat_importances.iloc[:20,:].plot(kind='barh',title = 'Feature Importance')

In [ ]:
feat_importances.iloc[-20:,:].plot(kind='barh',title = 'Feature Importance')